## Solution of a Damped Driven Pendulum using a Kolmogorov-Arnold Network

***Author: Brandon I. (brandon.ismalej.671@my.csun.edu)***

 NOTE: This notebook was written as a part of my assigned research duty and will be used as a component in the final deliverable. 